<a href="https://colab.research.google.com/github/kerryqliu/stats170/blob/irene/largedocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

from google.cloud import storage
from google.oauth2 import service_account

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import getpass

APIKEY = getpass.getpass()

··········


In [5]:
!pip install --upgrade pip

     |████████████████████████████████| 1.6MB 31.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [7]:
!pip install --upgrade google-api-python-client

     |████████████████████████████████| 7.1MB 4.3MB/s 
ERROR: earthengine-api 0.1.264 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 2.4.0 which is incompatible.
  Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [8]:
!pip install google-cloud-documentai

     |████████████████████████████████| 184kB 4.3MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 


In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/Shareddrives/CAPSTONE/Resources/analog-marking-308122-3287f19a2200.json"

In [2]:
!gcloud auth application-default print-access-token

ya29.c.Kp8B_weBA-LtJmYU8zR3tU8TLXlhWpg1MOfbzohS_UoNbLmm_z0sFarfI1jD0wRi0Bg0uJWgaSbtaf96fLrg190XmGVb3-8bOFDx1NAhmR-zlM8mo-mnRYGDwJqJN5uXNHTLO3jJHN094sXdoXA0DIz6elDQl3PgTrQ0mDd7txBfgwavh2I2_LpVhxZWrbni9P5Z71HTRcxiXhgZd8fbf90m


In [3]:
from google.cloud import documentai_v1beta2 as documentai

In [4]:
import re
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import re

from google.cloud import documentai_v1 as documentai
from google.cloud import storage

def batch_process_documents(
    project_id,
    location,
    processor_id,
    gcs_input_uri,
    gcs_output_uri,
    gcs_output_uri_prefix,
    timeout: int = 300,
):

    # You must set the api_endpoint if you use a location other than 'us', e.g.:
    opts = {}
    if location == "eu":
        opts = {"api_endpoint": "eu-documentai.googleapis.com"}

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    destination_uri = f"{gcs_output_uri}/{gcs_output_uri_prefix}/"

    gcs_documents = documentai.GcsDocuments(
        documents=[{"gcs_uri": gcs_input_uri, "mime_type": "application/pdf"}]
    )

    # 'mime_type' can be 'application/pdf', 'image/tiff',
    # and 'image/gif', or 'application/json'
    input_config = documentai.BatchDocumentsInputConfig(gcs_documents=gcs_documents)

    # Where to write results
    output_config = documentai.DocumentOutputConfig(
        gcs_output_config={"gcs_uri": destination_uri}
    )

    # Location can be 'us' or 'eu'
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"
    request = documentai.types.document_processor_service.BatchProcessRequest(
        name=name,
        input_documents=input_config,
        document_output_config=output_config,
    )

    operation = client.batch_process_documents(request)

    # Wait for the operation to finish
    operation.result(timeout=timeout)

    # Results are written to GCS. Use a regex to find
    # output files
    match = re.match(r"gs://([^/]+)/(.+)", destination_uri)
    output_bucket = match.group(1)
    prefix = match.group(2)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(output_bucket)
    blob_list = list(bucket.list_blobs(prefix=prefix))
    # print("Output files:")

    # for i, blob in enumerate(blob_list):
    #     # If JSON file, download the contents of this blob as a bytes object.
    #     if ".json" in blob.name:
    #         blob_as_bytes = blob.download_as_bytes()

    #         document = documentai.types.Document.from_json(blob_as_bytes)
    #         print(f"Fetched file {i + 1}")

    #         # For a full list of Document object attributes, please reference this page:
    #         # https://cloud.google.com/document-ai/docs/reference/rpc/google.cloud.documentai.v1beta3#document

    #         # Read the text recognition output from the processor
    #         for page in document.pages:
    #             for form_field in page.form_fields:
    #                 field_name = get_text(form_field.field_name, document)
    #                 field_value = get_text(form_field.field_value, document)
    #                 print("Extracted key value pair:")
    #                 print(f"\t{field_name}, {field_value}")
    #             for paragraph in document.pages:
    #                 paragraph_text = get_text(paragraph.layout, document)
    #                 print(f"Paragraph text:\n{paragraph_text}")
    #     else:
    #         print(f"Skipping non-supported file type {blob.name}")


# Extract shards from the text field
def get_text(doc_element: dict, document: dict):
    """
    Document AI identifies form fields by their offsets
    in document text. This function converts offsets
    to text snippets.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in doc_element.text_anchor.text_segments:
        start_index = (
            int(segment.start_index)
            if segment in doc_element.text_anchor.text_segments
            else 0
        )
        end_index = int(segment.end_index)
        response += document.text[start_index:end_index]
    return response


In [7]:
project_id= 'analog-marking-308122'
location = 'us' # Format is 'us' or 'eu'
processor_id = '71ad62c7e00b7da3' # Create processor in Cloud Console
gcs_output_uri = "gs://output-documents"

for i in range(139, 148):  #2,147):
  if i != 116:
    try:   
      gcs_input_uri = "gs://clinical-documents/clinicalexample" + str(i) + ".pdf"
      gcs_output_uri_prefix = "clinicalexample" + str(i)

      batch_process_documents(project_id,
        location,
        processor_id,
        gcs_input_uri,
        gcs_output_uri,
        gcs_output_uri_prefix)
      
      print(f"doc {i} is complete.")
    except:
      print(f"doc {i} failed.")

doc 139 failed.
doc 140 is complete.
doc 141 failed.
doc 142 is complete.
doc 143 is complete.
doc 144 is complete.
doc 145 is complete.
doc 146 is complete.
doc 147 is complete.


### database engine info

In [ ]:
from sqlalchemy import create_engine
# ENGINE

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### inserting documents into database

In [ ]:
import json

In [ ]:
for i in range(1,148):
  # figure out how many json pages are there
  pages = 0 # some number of pages
  current_text = ""
  for p in pages:

    filepath = '/content/drive/Shareddrives/CAPSTONE/documents/clinicalexample' + str(i) + '.pdf_7423456279627152130_0_clinicalexample' + str(i) + '-' + str(p) + '.json'
    with open(filepath) as f:
      data = json.load(f)
      current_text += ' ' + data["text"]

  engine.execute("INSERT INTO completeDocs VALUES (%s, %s)", i, current_text)

gs://medical-documents/clinicalexample116.pdf
gs://medical-documents/clinicalexample117.pdf
gs://medical-documents/clinicalexample118.pdf
gs://medical-documents/clinicalexample119.pdf
gs://medical-documents/clinicalexample120.pdf
gs://medical-documents/clinicalexample121.pdf
gs://medical-documents/clinicalexample122.pdf
gs://medical-documents/clinicalexample123.pdf
gs://medical-documents/clinicalexample124.pdf
gs://medical-documents/clinicalexample125.pdf
gs://medical-documents/clinicalexample126.pdf
gs://medical-documents/clinicalexample127.pdf
gs://medical-documents/clinicalexample128.pdf
gs://medical-documents/clinicalexample129.pdf
gs://medical-documents/clinicalexample130.pdf
gs://medical-documents/clinicalexample131.pdf
gs://medical-documents/clinicalexample132.pdf
gs://medical-documents/clinicalexample133.pdf
gs://medical-documents/clinicalexample134.pdf
gs://medical-documents/clinicalexample135.pdf
gs://medical-documents/clinicalexample136.pdf
gs://medical-documents/clinicalexa

In [ ]:
getDocs = "SELECT * from completeDocs;"
full_doc = pd.read_sql_query(getDocs, con=engine)
full_doc

,docid,fulltext
0,116.0,Document Text: From:\nVCU Health\nNeuroscience...
1,117.0,Document Text: FACSIMILE TRANSMITTAL SHEET\nFR...
2,118.0,Document Text: Date:\nHartford\nHealth Care\nA...
3,119.0,Document Text: MAYO\nCLINIC\nFax\nTo:\nDate:\n...
4,120.0,Document Text: Page 1 of 2\nClinical Summaries...
5,121.0,Document Text: Hospital for Special Care\nan a...
6,122.0,Document Text: briovaRx\nInfusion Services\nFR...
7,123.0,Document Text: Medical Records - CONFIDENTIAL\...
8,124.0,Document Text: FAX\nTO\nNumber of Pages (inclu...
9,125.0,Document Text: # 1/ 18\nTƠNTRAKT ANTARA Number...


In [ ]:
for index, row in full_doc.iterrows():
  print('-- document: ', row['docid'])
  tk = sent_tokenize(row['fulltext'])
  tk[0] = re.sub(r'Document Text: Example \d+\n', '', tk[0])
  for i in range(len(tk)):
    engine.execute("INSERT INTO docSentences VALUES (%s, %s, %s)", row['docid'], i+1, tk[i])

-- document:  116.0
-- document:  117.0
-- document:  118.0
-- document:  119.0
-- document:  120.0
-- document:  121.0
-- document:  122.0
-- document:  123.0
-- document:  124.0
-- document:  125.0
-- document:  126.0
-- document:  127.0
-- document:  128.0
-- document:  129.0
-- document:  130.0
-- document:  131.0
-- document:  132.0
-- document:  133.0
-- document:  134.0
-- document:  135.0
-- document:  136.0
-- document:  138.0
-- document:  140.0
-- document:  142.0
-- document:  143.0
-- document:  144.0
-- document:  145.0
-- document:  146.0
-- document:  147.0


In [ ]:
sent = "SELECT * from docSentences"
sent_df = pd.read_sql_query(sent, con=engine)
sent_df.tail(100)

,docid,sentencenum,sentence
9384,144.0,37.0,No history of stroke.
9385,144.0,38.0,MOTOR EXAM: muscle bulk was normal in all musc...
9386,145.0,1.0,Document Text: FAX COVER SHEET\nDATE:\nTO:\nPH...
9387,145.0,2.0,IF THE READER OF THIS FACSIMILE IS NOT THE INT...
9388,145.0,3.0,"IF YOU HAVE RECEIVED THIS FACSIMILE IN ERROR, ..."
...,...,...,...
9479,147.0,41.0,Homocysteine\n<11.4 umol/L\nVitamin B6\n2.1 21...
9480,147.0,42.0,4091\nP. 4\nA New To\nIn Network Benefits Only...
9481,147.0,43.0,4091\nP. 5\nMember Service\nPrecertification\n...
9482,147.0,44.0,Urgent Care Confinements\nmust call within 48 ...
